In [1]:
import torch
import torch.nn as nn
import time

## Load model

In [2]:
class FCN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(FCN, self).__init__()
        self.input_size = input_size
        self.l1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out) 
        return out

In [3]:
model = FCN(1,1,1)
model = torch.load('model_finetune.pth')
model

FCN(
  (l1): Linear(in_features=36, out_features=25, bias=True)
  (relu): ReLU()
  (l2): Linear(in_features=25, out_features=4, bias=True)
)

In [4]:
for weight in model.state_dict():
    print(weight)

l1.weight
l1.bias
l2.weight
l2.bias


In [5]:
for key, val in model.state_dict().items():
    print(key, val.shape)

l1.weight torch.Size([25, 36])
l1.bias torch.Size([25])
l2.weight torch.Size([4, 25])
l2.bias torch.Size([4])


## Print weights for HLS

In [6]:
l1_weight = model.state_dict()['l1.weight'].tolist()
l2_weight = model.state_dict()['l2.weight'].tolist()
l1_bias = model.state_dict()['l1.bias'].tolist()
l2_bias = model.state_dict()['l2.bias'].tolist()

In [7]:
print("const float l1_bias[HIDDEN_SIZE] = {", end='')
biases = ''
for b in l1_bias:
    biases += str(b) + ', '
biases = biases[:-2] + '};'
print(biases)

const float l1_bias[HIDDEN_SIZE] = {0.02950964868068695, 0.4142775535583496, 0.02706560492515564, 0.07311786711215973, 0.17308999598026276, 0.04190363362431526, 0.03293853998184204, 0.3943231701850891, 0.0506194643676281, 0.2176099270582199, 0.41307488083839417, -0.05125361308455467, -0.12481433153152466, -0.10703746229410172, 0.18086537718772888, 0.06666284054517746, 0.16100409626960754, 0.022622639313340187, 0.43974897265434265, 0.23154860734939575, 0.04789101332426071, -0.13133563101291656, -0.08701950311660767, 0.15997280180454254, -0.060387857258319855};


In [8]:
print("const float l2_bias[NUM_CLASSES] = {", end='')
biases = ''
for b in l2_bias:
    biases += str(b) + ', '
biases = biases[:-2] + '};'
print(biases)

const float l2_bias[NUM_CLASSES] = {0.05681278184056282, 0.2775717079639435, 0.06266991049051285, 0.23410113155841827};


In [9]:
print("const float l1_weights[HIDDEN_SIZE][INPUT_SIZE] = {", end='')
weights = ''
for row in l1_weight:
    weights += '{'
    for cell in row:
        weights += str(cell) + ', '
    weights = weights[:-2] + '},\n'
weights = weights[:-2] + '};'
print(weights)

const float l1_weights[HIDDEN_SIZE][INPUT_SIZE] = {{0.142061248421669, -1.0682300329208374, -0.6534576416015625, -0.2589521110057831, 0.3083900511264801, 0.16780543327331543, -0.11047011613845825, 0.2313552349805832, -0.0598272867500782, 0.014019707217812538, -0.05995134636759758, 0.09540297836065292, -0.17131313681602478, -0.08539816737174988, -0.01367239560931921, -0.1333649605512619, 0.1671353578567505, 0.20643705129623413, -0.0073904311284422874, -0.045938748866319656, 0.10352809727191925, 0.12977541983127594, 0.27678143978118896, 0.13544553518295288, -0.2660890519618988, -0.25170302391052246, -0.25871944427490234, -0.17832918465137482, -0.21421122550964355, -0.29739898443222046, 0.7568274140357971, -0.35063716769218445, -0.17316357791423798, 0.4436035752296448, -0.4882088005542755, -0.0965084508061409},
{0.1601371318101883, -0.28398534655570984, -0.09338650852441788, 0.3205435574054718, 0.31115227937698364, 0.30966365337371826, 0.5287277698516846, 0.40917471051216125, 0.1934950053

In [10]:
print("const float l2_weights[NUM_CLASSES][HIDDEN_SIZE] = {", end='')
weights = ''
for row in l2_weight:
    weights += '{'
    for cell in row:
        weights += str(cell) + ', '
    weights = weights[:-2] + '},\n'
weights = weights[:-2] + '};'
print(weights)

const float l2_weights[NUM_CLASSES][HIDDEN_SIZE] = {{-0.5440405607223511, -0.26513224840164185, 0.18948207795619965, 0.4581085443496704, 0.3680459260940552, -0.18766246736049652, -0.1601027548313141, -0.29499247670173645, 0.47193509340286255, -0.29295966029167175, -0.5657212734222412, 0.40705159306526184, -0.018943538889288902, 0.03976990655064583, -0.12431914359331131, -0.48011988401412964, 0.5912879705429077, 0.3933316171169281, -0.3485233187675476, 0.15046951174736023, 0.0204667616635561, -0.30370721220970154, 0.049600932747125626, 0.45058584213256836, 0.5910376310348511},
{0.3045894205570221, -0.6491499543190002, -0.5019345283508301, -0.06952328234910965, 0.2248215675354004, -0.3797239065170288, 0.07858555763959885, -0.7612782716751099, -0.5774893164634705, 0.3314230442047119, 0.07328079640865326, 0.021396854892373085, -0.032497938722372055, -0.23693180084228516, -0.5509703159332275, 0.5916024446487427, -0.5918861627578735, 0.06194145977497101, -0.6578897833824158, -0.8890674710273

## Get test data for software-hardware comparison

In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import preprocessing

In [12]:
TEST_DATA = 'test.csv'
TRAIN_DATA = 'train.csv'

In [13]:
test_df = pd.read_csv(TEST_DATA)

In [14]:
test_df = test_df[~test_df['Activity'].isin(['STANDING', 'WALKING'])]
features = [
    'tBodyAcc-mean()-X', 'tBodyAcc-mean()-Y', 'tBodyAcc-mean()-Z', 'tBodyAcc-std()-X',
    'tBodyAcc-std()-Y', 'tBodyAcc-std()-Z', 'tBodyAcc-max()-X', 'tBodyAcc-max()-Y',
    'tBodyAcc-max()-Z', 'tBodyAcc-min()-X', 'tBodyAcc-min()-Y', 'tBodyAcc-min()-Z',
    
    'tBodyGyro-mean()-X', 'tBodyGyro-mean()-Y', 'tBodyGyro-mean()-Z', 'tBodyGyro-std()-X',
    'tBodyGyro-std()-Y', 'tBodyGyro-std()-Z', 'tBodyGyro-max()-X', 'tBodyGyro-max()-Y',
    'tBodyGyro-max()-Z', 'tBodyGyro-min()-X', 'tBodyGyro-min()-Y', 'tBodyGyro-min()-Z',
    
    'tGravityAcc-std()-X', 'tGravityAcc-std()-Y', 'tGravityAcc-std()-Z', 'tGravityAcc-mad()-X',
    'tGravityAcc-mad()-Y', 'tGravityAcc-mad()-Z', 'tGravityAcc-max()-X', 'tGravityAcc-max()-Y',
    'tGravityAcc-max()-Z', 'tGravityAcc-min()-X', 'tGravityAcc-min()-Y', 'tGravityAcc-min()-Z',
    
    'Activity'
]

test_df = test_df[[f for f in features]]

In [15]:
X_test = test_df.iloc[:, :-1]
y_test = test_df[['Activity']]

In [16]:
le = preprocessing.LabelEncoder()
le.fit(y_test['Activity'].unique().tolist())
y_test = le.transform(y_test)

/Users/nguyenvanbinh/Desktop/hardware-ai/venv/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [17]:
np.unique(y_test)

array([0, 1, 2, 3])

In [18]:
np.where(y_test == 0)[0][1]

25

In [19]:
data = torch.Tensor(X_test.iloc[25].astype(float))

In [20]:
data

tensor([ 0.3009, -0.0236, -0.0969, -0.9868, -0.9749, -0.9869, -0.9247, -0.5651,
        -0.7979,  0.8371,  0.6710,  0.8461, -0.0315, -0.2453,  0.1566, -0.9872,
        -0.9511, -0.9415, -0.8678, -0.9630, -0.6845,  0.8389,  0.8598,  0.8170,
        -0.9754, -0.9854, -0.9765, -0.9753, -0.9849, -0.9763, -0.7833,  0.6752,
         0.6708, -0.6775,  0.7303,  0.6763])

In [23]:
with torch.no_grad():
    start_time = time.time()
    output = model(data)
    action = np.argmax(output)
    print(f"Inference time: {time.time() - start_time}")    
    print(action)

Inference time: 0.011629819869995117
tensor(0)
